# 형사법 데이터 ChromaDB 임베딩

**목표**: 388,767개의 형사법 판례 및 법령 청크를 ChromaDB에 임베딩

**데이터**:
- 판례 (HS_P): 32,525개 파일
- 법령 (HS_B): 798개 파일
- 총 청크: 388,767개
- 파일 크기: 456MB

**예상 소요 시간**: ~30-60분 (GPU 사용 시)

## 1. 패키지 설치

In [ ]:
# ChromaDB와 의존성 설치 (에러 무시)
!pip install -q chromadb sentence-transformers tqdm 2>&1 | grep -v "dependency conflicts" || true
print("✅ Packages installed (warnings ignored)")

## 2. 파일 업로드

`criminal_law_chunks.jsonl` 파일을 업로드하세요 (456MB)

In [ ]:
from google.colab import files
import os

# 파일 업로드
print("📁 Please upload criminal_law_chunks.jsonl")
uploaded = files.upload()

# 업로드된 파일 확인
if 'criminal_law_chunks.jsonl' in uploaded:
    file_size = os.path.getsize('criminal_law_chunks.jsonl') / (1024 * 1024)
    print(f"✅ File uploaded successfully: {file_size:.2f} MB")
else:
    print("❌ File not found. Please upload criminal_law_chunks.jsonl")

## 3. 임베딩 실행

In [ ]:
import json
import logging
from pathlib import Path
from typing import List, Dict
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch
import gc

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class OptimizedCriminalLawEmbedder:
    """최적화된 형사법 데이터 ChromaDB 임베딩 (스트리밍 + 체크포인트)"""

    def __init__(
        self,
        jsonl_file: str,
        chroma_db_path: str = "./chroma_db",
        collection_name: str = "criminal_law_docs",
        model_name: str = "jhgan/ko-sroberta-multitask",
        checkpoint_interval: int = 10000
    ):
        self.jsonl_file = Path(jsonl_file)
        self.chroma_db_path = Path(chroma_db_path)
        self.collection_name = collection_name
        self.checkpoint_interval = checkpoint_interval
        self.checkpoint_file = Path("embedding_checkpoint.txt")
        
        # GPU 감지 및 배치 크기 자동 조정
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        if self.device == "cuda":
            gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
            if gpu_mem > 30:  # V100, A100
                self.batch_size = 512
            elif gpu_mem > 14:  # T4
                self.batch_size = 256
            else:
                self.batch_size = 128
        else:
            self.batch_size = 64
        
        logger.info(f"🖥️  Device: {self.device.upper()}")
        logger.info(f"📦 Batch size: {self.batch_size}")

        # 임베딩 모델 로드
        logger.info(f"Loading embedding model: {model_name}")
        self.model = SentenceTransformer(model_name, device=self.device)
        logger.info("✅ Embedding model loaded")

        # ChromaDB 초기화
        logger.info(f"Initializing ChromaDB at {chroma_db_path}")
        self.client = chromadb.PersistentClient(
            path=str(self.chroma_db_path),
            settings=Settings(anonymized_telemetry=False)
        )

        # 컬렉션 가져오기 또는 생성
        try:
            self.collection = self.client.get_collection(name=self.collection_name)
            logger.info(f"📂 Existing collection found: {self.collection.count()} documents")
        except:
            self.collection = self.client.create_collection(
                name=self.collection_name,
                metadata={"description": "형사법 판례 및 법령 문서"}
            )
            logger.info(f"✅ New collection created: {self.collection_name}")

    def clean_metadata(self, metadata: Dict) -> Dict:
        """메타데이터에서 None 값 제거 및 타입 변환"""
        cleaned = {}
        for key, value in metadata.items():
            if value is None:
                continue  # None 값 스킵
            elif isinstance(value, (str, int, float, bool)):
                cleaned[key] = value
            elif isinstance(value, list):
                # 리스트는 문자열로 변환
                cleaned[key] = str(value)
            else:
                # 기타 타입은 문자열로 변환
                cleaned[key] = str(value)
        return cleaned

    def count_total_lines(self) -> int:
        """JSONL 파일의 총 라인 수 계산"""
        with open(self.jsonl_file, 'r', encoding='utf-8') as f:
            return sum(1 for _ in f)

    def load_checkpoint(self) -> int:
        """체크포인트에서 마지막 처리된 인덱스 로드"""
        if self.checkpoint_file.exists():
            with open(self.checkpoint_file, 'r') as f:
                last_index = int(f.read().strip())
                logger.info(f"📍 Resuming from checkpoint: {last_index}")
                return last_index
        return 0

    def save_checkpoint(self, index: int):
        """체크포인트 저장"""
        with open(self.checkpoint_file, 'w') as f:
            f.write(str(index))

    def embed_and_store_streaming(self):
        """스트리밍 방식으로 청크를 임베딩하고 ChromaDB에 저장"""
        # 총 라인 수 계산
        total_chunks = self.count_total_lines()
        logger.info(f"📊 Total chunks to process: {total_chunks:,}")
        
        # 체크포인트 로드
        start_index = self.load_checkpoint()
        if start_index > 0:
            logger.info(f"⏭️  Skipping first {start_index:,} chunks (already processed)")

        # 배치 버퍼
        batch_texts = []
        batch_ids = []
        batch_metadatas = []
        processed = start_index
        failed_chunks = []

        # 스트리밍 처리
        with open(self.jsonl_file, 'r', encoding='utf-8') as f:
            # 이미 처리된 라인 스킵
            for _ in range(start_index):
                next(f)
            
            # tqdm 진행률 표시
            with tqdm(total=total_chunks - start_index, initial=0, desc="Embedding") as pbar:
                for line_num, line in enumerate(f, start=start_index):
                    try:
                        chunk = json.loads(line)
                        batch_texts.append(chunk['text'])
                        batch_ids.append(chunk['id'])
                        # 메타데이터 클리닝
                        batch_metadatas.append(self.clean_metadata(chunk['metadata']))
                        
                        # 배치가 찼으면 임베딩 및 저장
                        if len(batch_texts) >= self.batch_size:
                            self._process_batch(batch_texts, batch_ids, batch_metadatas)
                            processed += len(batch_texts)
                            
                            # 배치 초기화
                            batch_texts = []
                            batch_ids = []
                            batch_metadatas = []
                            
                            # 진행률 업데이트
                            pbar.update(self.batch_size)
                            
                            # 체크포인트 저장
                            if processed % self.checkpoint_interval == 0:
                                self.save_checkpoint(processed)
                                logger.info(f"💾 Checkpoint saved at {processed:,} chunks")
                                
                                # 메모리 정리
                                gc.collect()
                                if self.device == "cuda":
                                    torch.cuda.empty_cache()
                    
                    except Exception as e:
                        logger.warning(f"⚠️  Failed to process chunk at line {line_num}: {str(e)[:200]}")
                        failed_chunks.append(line_num)
                        continue
                
                # 남은 배치 처리
                if batch_texts:
                    self._process_batch(batch_texts, batch_ids, batch_metadatas)
                    processed += len(batch_texts)
                    pbar.update(len(batch_texts))
        
        # 최종 체크포인트 저장
        self.save_checkpoint(processed)
        logger.info(f"✅ Embedding complete! Total processed: {processed:,}")
        
        if failed_chunks:
            logger.warning(f"⚠️  Failed chunks: {len(failed_chunks)}")
            with open('failed_chunks.txt', 'w') as f:
                f.write('\n'.join(map(str, failed_chunks)))

    def _process_batch(self, texts: List[str], ids: List[str], metadatas: List[Dict]):
        """배치 임베딩 및 저장"""
        try:
            # GPU로 직접 임베딩 생성
            embeddings = self.model.encode(
                texts,
                convert_to_tensor=True,
                show_progress_bar=False,
                device=self.device
            )
            
            # CPU로 변환 (ChromaDB 저장용)
            if isinstance(embeddings, torch.Tensor):
                embeddings = embeddings.cpu().numpy()
            
            # ChromaDB에 저장
            self.collection.add(
                ids=ids,
                embeddings=embeddings.tolist(),
                documents=texts,
                metadatas=metadatas
            )
        except Exception as e:
            logger.error(f"❌ Batch processing error: {e}")
            raise

    def verify_embedding(self):
        """임베딩 결과 검증"""
        count = self.collection.count()
        logger.info(f"📊 Total documents in ChromaDB: {count:,}")

        # 샘플 검색 테스트
        test_queries = [
            "음주운전 처벌",
            "절도죄 구성요건",
            "위법수집증거 배제"
        ]

        for test_query in test_queries:
            logger.info(f"\n🔍 Test query: '{test_query}'")
            
            query_embedding = self.model.encode([test_query], convert_to_tensor=True, device=self.device)
            if isinstance(query_embedding, torch.Tensor):
                query_embedding = query_embedding.cpu().numpy()
            
            results = self.collection.query(
                query_embeddings=query_embedding.tolist(),
                n_results=3
            )

            logger.info("Top 3 results:")
            for i, (doc, metadata) in enumerate(zip(results['documents'][0], results['metadatas'][0])):
                doc_type = metadata.get('type', 'unknown')
                source = metadata.get('source', 'unknown')
                case_num = metadata.get('case_number', 'N/A')
                logger.info(f"\n  {i+1}. [{doc_type}] {source}")
                logger.info(f"     Case: {case_num}")
                logger.info(f"     {doc[:150]}...")

    def run(self):
        """전체 임베딩 프로세스 실행"""
        logger.info("🚀 Starting optimized embedding process...")
        
        # 스트리밍 임베딩
        self.embed_and_store_streaming()
        
        # 검증
        self.verify_embedding()
        
        # 체크포인트 파일 삭제 (완료됨)
        if self.checkpoint_file.exists():
            self.checkpoint_file.unlink()
            logger.info("🗑️  Checkpoint file removed (embedding complete)")
        
        logger.info(f"\n✅ All done! ChromaDB saved at: {self.chroma_db_path}")
        logger.info(f"📦 You can now download the '{self.chroma_db_path}' folder")


# 실행
embedder = OptimizedCriminalLawEmbedder(
    jsonl_file="criminal_law_chunks.jsonl",
    chroma_db_path="./chroma_criminal_law",
    checkpoint_interval=10000  # 10,000개마다 체크포인트 저장
)

embedder.run()

## 4. ChromaDB 다운로드

임베딩이 완료되면 `chroma_criminal_law` 폴더를 압축하여 다운로드합니다.

In [ ]:
import shutil

# ChromaDB 폴더 압축
print("📦 Compressing ChromaDB folder...")
shutil.make_archive('chroma_criminal_law', 'zip', '.', 'chroma_criminal_law')

# 압축 파일 크기 확인
zip_size = os.path.getsize('chroma_criminal_law.zip') / (1024 * 1024)
print(f"✅ Compressed file size: {zip_size:.2f} MB")

# 다운로드
print("⬇️ Downloading...")
files.download('chroma_criminal_law.zip')
print("✅ Download complete!")

## 5. 로컬에서 사용하기

다운로드한 `chroma_criminal_law.zip` 파일을 로컬 프로젝트의 `data/vectordb/` 경로에 압축 해제하세요.

```bash
# 로컬 터미널에서
cd /path/to/lawlaw/data/vectordb
unzip chroma_criminal_law.zip
```

그러면 `/research/ai` 페이지에서 형사법 RAG 검색이 가능해집니다!